In [2]:
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os


#run number
yr=2019
run_no = 2
###Segment C.2
from UCModel import model
from UCModel import writeDatFile
from UCModel import readSolutionResult

def init_load(day,HorizonHours):
    df_load = pd.read_csv('../data/netload/CAISO2019Load.csv',header=0)[24*day:(24*day+HorizonHours)]
    # df_load = pd.read_csv('../data/netload/20200801CAISO.csv',header=0)[24*day:(24*day+HorizonHours)]

    return df_load

def init_import(day,HorizonHours):
    df_import = pd.read_csv('../data/electricityImport/CAISO2019Import.csv',header=0)[24*day:(24*day+HorizonHours)]
    return df_import

def init_export(day,HorizonHours):
    df_export = pd.read_csv('../data/electricityExport/CAISO2019Export.csv',header=0)[24*day:(24*day+HorizonHours)]

    return df_export

def init_gen():
    df_gen = pd.read_csv('../data/generator/GeneratorWOHydro2019.csv',header=0)
    return df_gen

def init_hydro(day):
    df_hydro = pd.read_csv('../data/hydro/HydroGenerator.csv',header=0)
    df_hydro_cap = pd.read_csv('../data/hydro/HydroDailyCapacity.csv',header=0)[day:day+1]
    cap=df_hydro_cap['Capacity(MWh)']
    df_hydro.at[0, 'maxcap_h'] = cap/24
    return df_hydro

def init_solar(day,HorizonHours):
    df_solar_cap=pd.read_csv('../data/generator/SolarCap.csv',header=0)[24*day:(24*day+HorizonHours)]
    df_solar_cons=pd.read_csv('../data/generator/SolarConstraint.csv',header=0)
    return  df_solar_cap,df_solar_cons

def init_wind(day,HorizonHours):
    df_wind_cap=pd.read_csv('../data/generator/WindCap.csv',header=0)[24*day:(24*day+HorizonHours)]
    df_wind_cons=pd.read_csv('../data/generator/WindConstraint.csv',header=0)
    return df_wind_cap,df_wind_cons

def init_veh(myresult):
    veh=pd.concat([myresult,myresult],axis=0)
    veh.index=range(veh.shape[0])
    return veh

def v2gResultConversion(myresult,vehicleNumber):
    N = 6# from 10mins to 1 hour
    myresult=myresult.groupby(myresult.index // N).sum()/6/1000000*vehicleNumber/100# from the unit of W to MW, from 100 vehicles to actual number of vehicles
    myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})
    return myresult

def saveResult(mwh_pd,myresult,versionName,day):
    mwh_pd.to_csv('../Result/validate'+str(day)+"_"+versionName+'vehicleGeneration.csv',index=False)


SimDays = 2
SimHours = SimDays * 24
HorizonHours = 48  ##planning horizon (e.g., 24, 48, 72 hours etc.)
regup_margin = 0.064  ##minimum regulation up reserve as a percent of system demand, referenced from https://www.nrel.gov/docs/fy19osti/73590.pdf
regdown_margin = 0.072 ##minimum regulation down reserve as a percent of system demand from https://www.nrel.gov/docs/fy19osti/73590.pdf
vehicleNumber=15000000
batteryCost=8
versionName='UCValidation_'+str(run_no)+'_battery'+str(batteryCost)
day=1
#initialize load
df_load=init_load(day,HorizonHours)
df_import=init_import(day,HorizonHours)
df_export=init_export(day,HorizonHours)
df_load['netload']=df_load['netload']-df_import['Import(MW)']+df_export['Export(MW)']
###initialize generators
df_gen=init_gen()
df_solar_cap,df_solar_cons=init_solar(day,HorizonHours)
df_wind_cap,df_wind_cons=init_wind(day,HorizonHours)
df_hydro=init_hydro(day)
# reindex
df_solar_cap.index=range(HorizonHours)
df_wind_cap.index=range(HorizonHours)
df_load.index=range(HorizonHours)



In [4]:
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os


#run number
yr=2019
run_no = 1
###Segment C.2
from UCModel import model
from UCModel import writeDatFile
from UCModel import readSolutionResult

def init_load(day,HorizonHours):
    df_load = pd.read_csv('../data/netload/CAISO2019Load.csv',header=0)[24*day:(24*day+HorizonHours)]
    # df_load = pd.read_csv('../data/netload/20200801CAISO.csv',header=0)[24*day:(24*day+HorizonHours)]

    return df_load

def init_import(day,HorizonHours):
    df_import = pd.read_csv('../data/electricityImport/CAISO2019Import.csv',header=0)[24*day:(24*day+HorizonHours)]
    return df_import

def init_export(day,HorizonHours):
    df_export = pd.read_csv('../data/electricityExport/CAISO2019Export.csv',header=0)[24*day:(24*day+HorizonHours)]

    return df_export

def init_gen():
    df_gen = pd.read_csv('../data/generator/GeneratorWOHydro2019.csv',header=0)
    return df_gen

def init_hydro(day):
    df_hydro = pd.read_csv('../data/hydro/HydroGenerator.csv',header=0)
    df_hydro_cap = pd.read_csv('../data/hydro/HydroDailyCapacity.csv',header=0)[day:day+1]
    cap=df_hydro_cap['Capacity(MWh)']
    df_hydro.at[0, 'maxcap_h'] = cap/24
    return df_hydro

def init_solar(day,HorizonHours):
    df_solar_cap=pd.read_csv('../data/generator/SolarCap.csv',header=0)[24*day:(24*day+HorizonHours)]
    df_solar_cons=pd.read_csv('../data/generator/SolarConstraint.csv',header=0)
    return  df_solar_cap,df_solar_cons

def init_wind(day,HorizonHours):
    df_wind_cap=pd.read_csv('../data/generator/WindCap.csv',header=0)[24*day:(24*day+HorizonHours)]
    df_wind_cons=pd.read_csv('../data/generator/WindConstraint.csv',header=0)
    return df_wind_cap,df_wind_cons

def init_veh(myresult):
    veh=pd.concat([myresult,myresult],axis=0)
    veh.index=range(veh.shape[0])
    return veh

def v2gResultConversion(myresult,vehicleNumber):
    N = 6# from 10mins to 1 hour
    myresult=myresult.groupby(myresult.index // N).sum()/6/1000000*vehicleNumber/100# from the unit of W to MW, from 100 vehicles to actual number of vehicles
    myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})
    return myresult

def saveResult(mwh_pd,myresult,versionName,day):
    mwh_pd.to_csv('../Result/validate'+versionName+"_"+str(day)+'_vehicleGeneration.csv',index=False)

SimDays = 2
SimHours = SimDays * 24
HorizonHours = 48  ##planning horizon (e.g., 24, 48, 72 hours etc.)
regup_margin = 0.064  ##minimum regulation up reserve as a percent of system demand, referenced from https://www.nrel.gov/docs/fy19osti/73590.pdf
regdown_margin = 0.072 ##minimum regulation down reserve as a percent of system demand from https://www.nrel.gov/docs/fy19osti/73590.pdf
vehicleNumber=15000000
batteryCost=8
versionName='UCValidation_'+str(run_no)+'_battery'+str(batteryCost)

day=363
#initialize load
df_load=init_load(day,HorizonHours)
# print(df_load)

df_import=init_import(day,HorizonHours)
# print(df_import)

df_export=init_export(day,HorizonHours)
# print(df_export)

df_load['netload']=df_load['netload']-df_import['Import(MW)']+df_export['Export(MW)']



In [5]:
df_load['netload'].sum()

735644.1250066666

In [11]:
df_import = pd.read_csv('../data/electricityImport/CAISO2019Import.csv',header=0)
print(df_import['Import(MW)'].sum())
df_load = pd.read_csv('../data/netload/CAISO2019Load.csv',header=0)
print(df_load['netload'].sum())
df_export = pd.read_csv('../data/electricityExport/CAISO2019Export.csv',header=0)
print(df_export['Export(MW)'].sum())
df_load['netload']=df_load['netload']-df_import['Import(MW)']+df_export['Export(MW)']
df_load['netload'].sum()



238319563.869187
219456619
171880038.0625


153025779.058303

In [8]:
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os

day=1
df_hydro = pd.read_csv('../data/hydro/HydroGenerator.csv',header=0)

df_hydro_cap = pd.read_csv('../data/hydro/HydroDailyCapacity.csv',header=0)[day-1:day]
cap=df_hydro_cap['Capacity(MWh)']
df_hydro.at[0, 'maxcap_h'] = cap
print(df_hydro)


name  maxcap_h  mincap_h  opcost_h  minup_h    ramp_h  st_cost_h  \
0  hydro     44157         0  0.024079        0  23.59814          0   

   var_om_h  fix_om_h  regcost_h  
0         0     14130       9999  


In [1]:
from __future__ import division
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
from datetime import datetime
import os
import datetime as dt
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import v2gsim
import matplotlib.pyplot as plt
import seaborn as sns

day=0

df_hydro = pd.read_csv('../data/hydro/HydroGenerator.csv',header=0)
df_hydro_cap = pd.read_csv('../data/hydro/HydroDailyCapacity.csv',header=0)[day:day+1]['Capacity(MWh)']
df_hydro['maxcap']=float(df_hydro_cap)



KeyError: 0

In [4]:
import pandas as pd 
SimDays = 2
SimHours = SimDays * 24
HorizonHours = 48  ##planning horizon (e.g., 24, 48, 72 hours etc.)
regup_margin = 0.064  ##minimum regulation up reserve as a percent of system demand, referenced from https://www.nrel.gov/docs/fy19osti/73590.pdf
regdown_margin = 0.072 ##minimum regulation down reserve as a percent of system demand from https://www.nrel.gov/docs/fy19osti/73590.pdf
vehicleNumber=15000000
batteryCost=8
day=0
yr=2019
run_no = 3
versionName='UCValidation_'+str(run_no)+'_battery'+str(batteryCost)


In [29]:

mwh_pd=pd.read_csv('../Result/validate'+str(day)+"_"+versionName+'vehicleGeneration.csv')

mwh=mwh_pd.groupby('Generator').sum()
genFleet=pd.read_csv('../data/generator/GeneratorRaw2019.csv',header=0)
mwh=mwh.merge(genFleet[['name','PlantType']],left_on='Generator',right_on='name',how='left')
mwh.at[367,'PlantType']='Solar'
mwh.at[368,'PlantType']='Vehicle'
mwh.at[369,'PlantType']='Wind'
mwh.at[367,'name']='Solar'
mwh.at[368,'name']='Vehicle'
mwh.at[369,'name']='Wind'
mwh.groupby('PlantType').sum()

mwh.tail()

,Day,Hour,Generation,Reg Up,Reg Down,name,PlantType
365,25,325,0.000000,0.00,0.00,7552_G_CCST,Combined Cycle
366,25,325,243.160000,6.84,143.16,902_G_1,Geothermal
367,25,325,527.491380,0.00,0.00,Solar,Solar
368,50,650,45927.803721,0.00,0.00,Vehicle,Vehicle
369,25,325,18949.463360,0.00,0.00,Wind,Wind


In [1]:
import pandas as pd
import os

def GetData(version):
    directory = r'/Users/meiyewang/Desktop/code/Result'
    df1=pd.DataFrame(columns=["Generator","Day","Hour","Generation","Reg Up","Reg Down"])
    for filename in os.listdir(directory):
        if filename.startswith(version) :
            # if filename.split('_')[2]==str(run_no):
            df=pd.read_csv(os.path.join(directory, filename))
            df1=pd.concat([df1,df[["Generator","Day","Hour","Generation","Reg Up","Reg Down"]]])
        else:
            continue
    return df1

def aggregateGeneration(df):
    mwh=df.groupby('Generator').sum()
    genFleet=pd.read_csv('../data/generator/GeneratorRaw2019.csv',header=0)
    mwh=mwh.merge(genFleet[['name','PlantType']],left_on='Generator',right_on='name',how='left')
    mwh.at[367,'PlantType']='Hydro'
    mwh.at[368,'PlantType']='Solar'
    mwh.at[370,'PlantType']='Wind'
    mwh.at[367,'name']='Hydro'
    mwh.at[368,'name']='Solar'
    mwh.at[370,'name']='Wind'
    aggregateGeneration=mwh.groupby('PlantType').sum()
    return aggregateGeneration

def percentage(aggregateGeneration):
    
    generation=aggregateGeneration['Generation'].sum()
    aggregateGeneration['percentage']= aggregateGeneration['Generation']/ generation
    print(aggregateGeneration)

df=GetData("validateUCValidation_1")
aggregateGeneration=aggregateGeneration(df)
percentage(aggregateGeneration)
aggregateGeneration.to_csv('ValidationResult.csv')

Generation        Reg Up      Reg Down  percentage
PlantType                                                                  
Biomass                5.199748e+06  0.000000e+00  0.000000e+00    0.023454
Coal Steam             1.508750e+05  0.000000e+00  9.052500e+04    0.000681
Combined Cycle         9.404930e+07  1.358949e+07  5.977830e+06    0.424227
Combustion Turbine     2.546340e+06  3.352000e-02  0.000000e+00    0.011486
Fossil Waste           4.583094e+05  0.000000e+00  0.000000e+00    0.002067
Fuel Cell              3.828345e+02  0.000000e+00  0.000000e+00    0.000002
Geothermal             1.623060e+07  0.000000e+00  9.738360e+06    0.073211
Hydro                  3.880894e+07  0.000000e+00  0.000000e+00    0.175055
Landfill Gas           2.437900e+06  0.000000e+00  0.000000e+00    0.010997
Municipal Solid Waste  3.747266e+05  0.000000e+00  0.000000e+00    0.001690
Non-Fossil Waste       1.168771e+06  0.000000e+00  0.000000e+00    0.005272
Nuclear                1.953124e+07  

In [6]:
aggregateGeneration['Generation'].sum()

162931354.56689364

In [3]:
aggregateGeneration.to_csv('ValidationResult.csv')

In [60]:
def aggregateGeneration(df):
    mwh=df.groupby('Generator').sum()
    genFleet=pd.read_csv('../data/generator/GeneratorRaw2019.csv',header=0)
    mwh=mwh.merge(genFleet[['name','PlantType']],left_on='Generator',right_on='name',how='left')
    mwh.at[367,'PlantType']='Solar'
    mwh.at[368,'PlantType']='Vehicle'
    mwh.at[369,'PlantType']='Wind'
    mwh.at[367,'name']='Solar'
    mwh.at[368,'name']='Vehicle'
    mwh.at[369,'name']='Wind'
    aggregateGeneration=mwh.groupby('PlantType').sum()
    return aggregateGeneration
aggregateGeneration()

,Generation,Reg Up,Reg Down
PlantType,,,
Biomass,5.892028e+06,0.000000e+00,0.000000e+00
Coal Steam,1.462607e+05,7.487316e+03,1.229912e+04
Combined Cycle,6.820457e+07,1.025770e+07,2.106336e+05
Combustion Turbine,1.049770e+06,0.000000e+00,0.000000e+00
Fossil Waste,5.159998e+05,0.000000e+00,0.000000e+00
Fuel Cell,0.000000e+00,0.000000e+00,0.000000e+00
Geothermal,1.654193e+07,4.101209e+03,1.178745e+07
Landfill Gas,2.810765e+06,0.000000e+00,0.000000e+00
Municipal Solid Waste,4.158209e+05,0.000000e+00,0.000000e+00


In [73]:
mwh_group=mwh.groupby('PlantType').sum()
generation=mwh_group['Generation'].sum()
# mwh_group
coal=mwh_group.at['Coal Steam','Generation']/200475000
print(coal)
ng=(mwh_group.at['Combined Cycle','Generation']+mwh_group.at['Combustion Turbine','Generation'])/200475000
print(ng)
nuclear=(mwh_group.at['Nuclear','Generation'])/200475000
print(nuclear)
renewable=(mwh_group.at['Solar','Generation']+mwh_group.at['Wind','Generation']+mwh_group.at['Vehicle','Generation'])/200475000
print(renewable)
biomass=(mwh_group.at['Biomass','Generation'])/200475000
print(biomass)
geothermal=(mwh_group.at['Geothermal','Generation'])/200475000
print(geothermal)
wind=(mwh_group.at['Wind','Generation'])/200475000
print(wind)

0.00072957068914661
0.3454512429481703
0.09945689815383467
0.24930540900404768
0.02939033565823097
0.08251367887928089
0.049683277102905744


In [80]:
other=(mwh_group.at['Landfill Gas','Generation']+mwh_group.at['Municipal Solid Waste','Generation']+mwh_group.at['Non-Fossil Waste','Generation'])/360*365
print(other)



4609895.408632993


In [5]:
mwh=df.groupby('Generator').sum()
# genFleet=pd.read_csv('../data/generator/GeneratorRaw2019.csv',header=0)
# mwh=mwh.merge(genFleet[['name','PlantType']],left_on='Generator',right_on='name',how='left')
mwh

,Generation,Reg Up,Reg Down
Generator,,,
10034_G_GEN1,2.227000e+03,0.000000,0.000000
10034_G_GEN2,9.720000e+02,0.000000,0.000000
10052_B_BLR1,5.569958e+05,0.000000,0.000000
10091_G_G2,8.769439e+04,0.000000,0.000000
10091_G_GEN1,3.473256e+04,0.000000,0.000000
10110_G_GEN1,3.076548e+02,0.000000,0.000000
10144_B_BLR3,5.001870e+04,0.000000,0.000000
10156_G_GEN2,1.308000e+02,0.000000,0.000000
10156_G_GEN3,3.003460e+02,0.000000,0.000000


In [10]:
day=0

df_hydro = pd.read_csv('../data/hydro/HydroGenerator.csv',header=0)
df_hydro_cap = pd.read_csv('../data/hydro/HydroDailyCapacity.csv',header=0)[day:day+1]['Capacity(MWh)']
df_hydro[0:1]['maxcap']=float(df_hydro_cap)
df_hydro



,name,maxcap,mincap,opcost,minup,ramp,st_cost,var_om,fix_om,regcost
0,hydro,0,0,0.024079,0,23.59814,0,0,14130,9999


In [ ]:

writeDatFile(df_gen,df_solar_cap,df_solar_cons,df_wind_cap,df_wind_cons,df_veh_cap,df_load,SimDays,SimHours,HorizonHours,regup_margin,regdown_margin,versionName,batteryCost)
# print(df_veh_cap)
###Create instand and solve 
instance = model.create_instance(versionName+'.dat')
opt = SolverFactory("gurobi") ##SolverFactory("cplex")
# opt.options["threads"] = 1
H = instance.HorizonHours
K=range(1,H+1)

pr2,vehCap2=readSolutionResult(opt,instance,K,veh,df_gen,df_solar_cons,df_wind_cons,batteryCost)
print(pr2)
print("complete")

In [ ]:
def init_gen():
    df_gen = pd.read_csv('../data/generator/GeneratorRaw2019.csv',header=0)
    return df_gen
df_gen=init_gen().groupby('PlantType').sum()
df_gen

In [ ]:
df_load_total = pd.read_csv('../data/netload/CAISO2019Load.csv',header=0)
df_load_total.max()

In [ ]:

df_load=init_load(day,HorizonHours)
df_load.max()

In [ ]:
def init_gen():
    df_gen = pd.read_csv('../data/generator/Generator.csv',header=0)
    return df_gen
init_gen().sum()   

In [87]:
df_solar_cap=pd.read_csv('../data/generator/SolarCap.csv',header=0)
df_solar_cap.sum()/1000

MW    26609.433766
dtype: float64

In [89]:
df_solar_cap.shape

(8760, 1)

In [88]:
df_wind_cap=pd.read_csv('../data/generator/WindCap.csv',header=0)
df_wind_cap.sum()/1000


MW    13886.920096
dtype: float64

In [5]:
day=0
HorizonHours=48

load= pd.read_csv('../data/netload/CAISO2019Load.csv',header=0)

import datetime as dt
load['Unnamed: 0'] = [dt.datetime.strptime(x,"%Y-%m-%d %H:%M") for x in load['Unnamed: 0']]
load=load.set_index(load['Unnamed: 0'])
load=load.drop(columns='Unnamed: 0')
load.to_csv('../data/netload/CAISO2019LoadTimeAsIndex.csv')

In [6]:
load_com = pd.read_csv('../data/netload/CAISO2019LoadTimeAsIndex.csv',header=0)
print(load_com)

Unnamed: 0  netload
0     2019-01-01 00:00:00    21934
1     2019-01-01 01:00:00    21188
2     2019-01-01 02:00:00    20613
3     2019-01-01 03:00:00    20298
4     2019-01-01 04:00:00    20338
5     2019-01-01 05:00:00    20807
6     2019-01-01 06:00:00    21543
7     2019-01-01 07:00:00    21514
8     2019-01-01 08:00:00    20906
9     2019-01-01 09:00:00    20291
10    2019-01-01 10:00:00    19691
11    2019-01-01 11:00:00    19097
12    2019-01-01 12:00:00    18680
13    2019-01-01 13:00:00    18765
14    2019-01-01 14:00:00    19406
15    2019-01-01 15:00:00    20621
16    2019-01-01 16:00:00    23016
17    2019-01-01 17:00:00    26277
18    2019-01-01 18:00:00    26921
19    2019-01-01 19:00:00    26736
20    2019-01-01 20:00:00    26319
21    2019-01-01 21:00:00    25351
22    2019-01-01 22:00:00    23802
23    2019-01-01 23:00:00    22274
24    2019-01-02 00:00:00    21285
25    2019-01-02 01:00:00    20594
26    2019-01-02 02:00:00    20275
27    2019-01-02 03:00:00    20401


In [2]:
import pandas as pd
day=0
HorizonHours=48
import datetime as dt

def init_loadWithIndex(load_com,day,HorizonHours):
    load = load_com[24*day:(24*day+HorizonHours)]
    return load

load=init_loadWithIndex(load_com,day,HorizonHours)[:25]
print(load)

Unnamed: 0  netload
0   2019-01-01 00:00:00    21934
1   2019-01-01 01:00:00    21188
2   2019-01-01 02:00:00    20613
3   2019-01-01 03:00:00    20298
4   2019-01-01 04:00:00    20338
5   2019-01-01 05:00:00    20807
6   2019-01-01 06:00:00    21543
7   2019-01-01 07:00:00    21514
8   2019-01-01 08:00:00    20906
9   2019-01-01 09:00:00    20291
10  2019-01-01 10:00:00    19691
11  2019-01-01 11:00:00    19097
12  2019-01-01 12:00:00    18680
13  2019-01-01 13:00:00    18765
14  2019-01-01 14:00:00    19406
15  2019-01-01 15:00:00    20621
16  2019-01-01 16:00:00    23016
17  2019-01-01 17:00:00    26277
18  2019-01-01 18:00:00    26921
19  2019-01-01 19:00:00    26736
20  2019-01-01 20:00:00    26319
21  2019-01-01 21:00:00    25351
22  2019-01-01 22:00:00    23802
23  2019-01-01 23:00:00    22274
24  2019-01-02 00:00:00    21285


In [8]:
import pandas as pd
from datetime import datetime
import numpy as np
import v2gsim
vehCap=pd.read_csv('../Result/2019_1_battery_8_renewable_0.7_vehicleNum_1500000_0_vehicleGeneration.csv',header=0)
pr=pd.read_csv    ('../Result/2019_1_battery_8_renewable_0.7_vehicleNum_1500000_0__price.csv',header=0)
# myresult=v2gResultConversion(pd.read_csv('../data/vehicle/VehiclesCap_V2G_0_2019_1_battery8_renewable0.7_vehicleNum1500000.csv',header=0),vehicleNumber)     

vehCap=pd.concat([vehCap,vehCap],ignore_index=True)
pr=pd.concat([pr,pr],ignore_index=True)
vehCap
# vehCap.iloc[-1]

,Generator,Day,Hour,gen_capacity_veh,regup_capacity_veh,regdown_capacity_veh
0,vehicle,1,1,5296.692301,0.0,5200.483010
1,vehicle,1,2,5305.586535,0.0,5210.519415
2,vehicle,1,3,5306.481817,0.0,5270.051250
3,vehicle,1,4,5344.364082,0.0,5072.007917
4,vehicle,1,5,5222.595872,0.0,5041.634958
5,vehicle,1,6,5049.000000,0.0,4871.920710
6,vehicle,1,7,4917.000000,0.0,4836.597593
7,vehicle,1,8,4372.500000,0.0,4600.914658
8,vehicle,1,9,4636.500000,0.0,4749.725333
9,vehicle,1,10,4504.769756,0.0,4772.341130


In [3]:
pr


,0,Hour,pr_e,pr_fre_d,pr_fre_u
0,NaN,0.0,31.999246,8.0,2.686903
1,NaN,1.0,31.999246,8.0,2.684355
2,NaN,2.0,31.999246,8.0,2.667639
3,NaN,3.0,31.999246,8.0,2.666075
4,NaN,4.0,31.999246,8.0,2.666851
5,NaN,5.0,31.999246,8.0,2.684735
6,NaN,6.0,31.999246,8.0,2.691150
7,NaN,7.0,31.999246,8.0,2.687091
8,NaN,8.0,31.999246,8.0,2.659590
9,NaN,9.0,31.999246,8.0,2.542985
